Импортируем нужные модули

In [14]:
import csv
import re
from nltk.tokenize import WordPunctTokenizer

Взял текст с сайта зе флоу, объясняю почему он: много сложных случаев таких как имена артистов всякие жаргонизмы и прочее. В тексте фигурирует множество дат и указаний на время, а еще тут много английских слов. 


In [15]:
text = '''
В огромном интервью Олег Савченко вспоминает всё: как его карьера чуть было не закончилась, даже не успев начаться; как он писал песни про магию в тамбурах поездов и комнатах студенческих общежитий, как состоял в одном рэп-объединении с Максом Коржом, и как потерял одну из песен, написанных для этого релиза (но он нам ее напел; как говорят в таких случаях: worldwide exclusive!).

Олег Савченко вспоминает, как вдохновлялся "хипстер-хопом" (кто сегодня вспомнит, что это вообще такое?), искал свою формулу и "начал подозревать, что читать необязательно"; как написал альбом и просто отправил на 3 почты — и вдруг о нем заговорили Афиша и Rap.ru, а снятый за копейки клип показали на ТВ. Что было дальше, вы знаете сами.

В 2007-м я поступил в универ и переехал из Витебска в Минск. Я уже знал про минскую тусу M.U Skool, где собирались самые крутые рэперы Беларуси. Поначалу это была тусовка из 10 человек, среди них — локальные рэп-герои Deech, Вожык, Zeman, Reezah Realize и Сашок, чувак, который всю эту херню двигал и спонсировал. Из кружка по интересам это превратилось в тусы, куда в пиковый период приходило по 200 человек.

Местом сбора была станция метро Спортивная. По субботам в 19:00 все туда съезжались, закупались пивасом, чипсами, рыбой, и шли в дальний переход. Обязательно был кто-то с бумбоксом, ставили только биты конца 90-х. Все, что с рэпом происходило позже, старались не замечать. 

'''

Сначала токенизиру текст в цсв файл а затем руками размечаю каждый токен при помози экселя затем импортирую обратно из формата хслс в цсв

In [16]:
tokens = WordPunctTokenizer().tokenize(text)

In [17]:
len(tokens),tokens[:5]

(289, ['В', 'огромном', 'интервью', 'Олег', 'Савченко'])

In [5]:
with open('corpus.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for w in tokens:
        writer.writerow([w, ' '])

по итогу у нас получается файл с золотым стандартом with_tags.csv

In [192]:
tagged = []
with open('with_tags.csv', 'r') as csv_file:
    reader = csv.reader(csv_file, delimiter=',')
    for row in reader:
        if len(row) == 2:
            tagged.append((row[0], row[1]))

In [193]:
len(tagged), tagged[:5]

(264,
 [('В', 'ADP'),
  ('огромном', 'ADJ'),
  ('интервью', 'NOUN'),
  ('Олег', 'PROPN'),
  ('Савченко', 'PROPN')])

Теперь для того чтобы сравнить золотой стандарт с другими морфоанализаторами, и понять насколько они точные, надо выбрать эти самые морфоанализаторы и распарситьт с помощью них текст. В своей работе я возьму пайморфи станзу и наташу

Начну с наташи

In [30]:
!pip3 install natasha

     |████████████████████████████████| 34.4 MB 126 kB/s eta 0:00:01
     |████████████████████████████████| 49 kB 178 kB/s eta 0:00:01
     |████████████████████████████████| 41 kB 88 kB/s  eta 0:00:01
Using legacy 'setup.py install' for intervaltree, since package 'wheel' is not installed.
    Running setup.py install for intervaltree ... done


In [31]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    Doc
)
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
segmenter = Segmenter()

In [32]:
text_n = Doc(text)

In [33]:
text_n.segment(segmenter)

In [37]:
text_n.tag_morph(morph_tagger)

In [40]:
text_with_natasha = [(_.text, _.pos) for _ in text_n.tokens]

In [42]:
text_with_natasha[:5]

[('В', 'ADP'),
 ('огромном', 'ADJ'),
 ('интервью', 'NOUN'),
 ('Олег', 'PROPN'),
 ('Савченко', 'PROPN')]

In [94]:
with open('natasha.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for c in text_with_natasha:
        writer.writerow([c[0], c[1]])

In [178]:
ntsh = []
with open('natasha.csv', 'r', encoding = 'UTF-8') as csv_file:
    reader = csv.reader(csv_file, delimiter=',')
    for row in reader:
        if len(row) == 2:
            ntsh.append((row[0], row[1]))

In [179]:
ntsh[:5]

[('В', 'ADP'),
 ('огромном', 'ADJ'),
 ('интервью', 'NOUN'),
 ('Олег', 'PROPN'),
 ('Савченко', 'PROPN')]

Станза

In [43]:
!pip3 install stanza

     |████████████████████████████████| 342 kB 76 kB/s eta 0:00:011
     |████████████████████████████████| 218.8 MB 2.2 MB/s eta 0:00:012   |█▋                              | 11.2 MB 289 kB/s eta 0:11:58     |██████▉                         | 46.6 MB 1.9 MB/s eta 0:01:31
     |████████████████████████████████| 1.0 MB 1.2 MB/s eta 0:00:01


In [44]:
import stanza
stanza.download('ru')

2021-10-02 14:41:29 INFO: Downloading default packages for language: ru (Russian)...


2021-10-02 14:51:26 INFO: Finished downloading models and saved to /Users/tarasandrushko/stanza_resources.


In [50]:
nlp = stanza.Pipeline(lang='RU', processors='tokenize,pos')

2021-10-02 14:55:24 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |

2021-10-02 14:55:24 INFO: Use device: cpu
2021-10-02 14:55:24 INFO: Loading: tokenize
2021-10-02 14:55:24 INFO: Loading: pos
2021-10-02 14:55:24 INFO: Done loading processors!


In [53]:
doc = nlp(text)

In [161]:
text_with_stanza = []
for sent in doc.sentences:
    for word in sent.words:
        text_with_stanza.append((word.text, word.upos))

In [162]:
text_with_stanza[:5]

[('В', 'ADP'),
 ('огромном', 'ADJ'),
 ('интервью', 'NOUN'),
 ('Олег', 'PROPN'),
 ('Савченко', 'PROPN')]

In [166]:
with open('stanza.csv', 'w', newline='', encoding = 'UTF-8') as csvfile:
    writer = csv.writer(csvfile)
    for s in text_with_stanza:
        writer.writerow([s[0], s[1]])

In [171]:
stnz = []
with open('stanza.csv', 'r', encoding = 'UTF-8') as csv_file:
    reader = csv.reader(csv_file, delimiter=',')
    for row in reader:
        if len(row) == 2:
            stnz.append((row[0], row[1]))

In [172]:
stnz[:5]

[('В', 'ADP'),
 ('огромном', 'ADJ'),
 ('интервью', 'NOUN'),
 ('Олег', 'PROPN'),
 ('Савченко', 'PROPN')]

Pymorphy

In [63]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [65]:
pymorphy_lib = []
with open('with_tags.csv', 'r') as csv_file:
    reader = csv.reader(csv_file, delimiter=';')
    for row in reader:
        if len(row) == 2:
            pymorphy_lib.append(row[0])

In [67]:
pymorphy_with_tags = []
for lib in pymorphy_lib:
    parse = morph.parse(lib)[0]
    pos = parse.tag.POS
    pymorphy_with_tags.append((lib, pos))

In [70]:
pymorphy_with_tags[:5]

[('В', 'PREP'),
 ('огромном', 'ADJF'),
 ('интервью', 'NOUN'),
 ('Олег', 'NOUN'),
 ('Савченко', 'NOUN')]

In [71]:
with open('pymorphy.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for column in text_with_stanza:
        writer.writerow([column[0], column[1]])

Сравним точность парсеров с золотым стандартом при помощи функции

In [229]:
accuracies = []
tagged = []
with open('with_tags.csv', 'r') as csv_file:
    reader = csv.reader(csv_file, delimiter=',')
    for row in reader:
        if len(row) == 2:
            tagged.append((row[0], row[1]))
            
def get_accuracy(file):
    with open(file, 'r') as csv_file:
        corp = []
        reader = csv.reader(csv_file, delimiter=',')
        for row in reader:
            if len(row) == 2:
                corp.append((row[0], row[1]))
    
    for tc in range(len(tagged)):
        check = 0
        if tagged[tc][0] == corp[tc][0]:
            if tagged[tc][1] == corp[tc][1]:
                accuracies.append(1)
                check = 1
            else:
                accuracies.append(0)
                check = 1
        else:
            for c in corp:
                if c[0] == tagged[tc][0]:
                    if c[1] == tagged[tc][1]:
                        accuracies.append(1)
                        check = 1
                        break
        if check == 0:
            accuracies.append(0)
    return accuracies
        

In [230]:
print(sum(get_accuracy('stanza.csv'))/len(get_accuracy('stanza.csv')))

0.4053030303030303


In [231]:
print(sum(get_accuracy('natasha.csv'))/len(get_accuracy('natasha.csv')))

0.5975378787878788


Не очень точно от станзы, если честно даже обидно что пришлось для этого скачать гиг....



Теперь чанкиЪ сделаю для наташи тк она точнее оказалась

In [243]:
def chunckies(corpus):
    noun = []
    verb = []
    vnoun = []
    for c in range(len(corpus) - 1):
        if corpus[c][1] == 'ADJ' and corpus[c+1][1] == 'NOUN':
            noun.append((corpus[c][0], corpus[c+1][0]))
        if corpus[c][1] == 'NOUN' and corpus[c+1][1] == 'VERB':
            verb.append((corpus[c][0], corpus[c+1][0]))
        if corpus[c][0].lower() == 'в' and corpus[c+1][1] == 'NOUN':
            vnoun.append((corpus[c][0], corpus[c+1][0]))
    return noun, verb, vnoun

In [244]:
natasha = []
with open('natasha.csv', 'r') as csv_file:
    reader = csv.reader(csv_file, delimiter=',')
    for row in reader:
        if len(row) == 2:
            natasha.append((row[0], row[1]))
noun, verb, vnoun = chunckies(natasha)

In [245]:
noun

[('огромном', 'интервью'),
 ('студенческих', 'общежитий'),
 ('минскую', 'тусу'),
 ('крутые', 'рэперы'),
 ('локальные', 'рэп-герои'),
 ('пиковый', 'период'),
 ('дальний', 'переход')]

In [246]:
verb

[('клип', 'показали'), ('период', 'приходило'), ('рэпом', 'происходило')]

In [247]:
vnoun

[('в', 'тамбурах'), ('в', 'универ'), ('в', 'тусы')]